# **Final Project**

##**Problem stament :**     

The widespread dissemination of fake news and propaganda presents serious societal risks, including the erosion of public trust, political polarization, manipulation of elections, and the spread of harmful misinformation during crises such as pandemics or conflicts. From an NLP perspective, detecting fake news is fraught with challenges. Linguistically, fake news often mimics the tone and structure of legitimate journalism, making it difficult to distinguish using surface-level features. The absence of reliable and up-to-date labeled datasets, especially across multiple languages and regions, hampers the effectiveness of supervised learning models. Additionally, the dynamic and adversarial nature of misinformation means that malicious actors constantly evolve their language and strategies to bypass detection systems. Cultural context, sarcasm, satire, and implicit bias further complicate automated analysis. Moreover, NLP models risk amplifying biases present in training data, leading to unfair classifications and potential censorship of legitimate content. These challenges underscore the need for cautious, context-aware approaches, as the failure to address them can inadvertently contribute to misinformation, rather than mitigate it.



Use datasets in link : https://drive.google.com/drive/folders/1mrX3vPKhEzxG96OCPpCeh9F8m_QKCM4z?usp=sharing
to complete requirement.

## **About dataset:**

* **True Articles**:

  * **File**: `MisinfoSuperset_TRUE.csv`
  * **Sources**:

    * Reputable media outlets like **Reuters**, **The New York Times**, **The Washington Post**, etc.

* **Fake/Misinformation/Propaganda Articles**:

  * **File**: `MisinfoSuperset_FAKE.csv`
  * **Sources**:

    * **American right-wing extremist websites** (e.g., Redflag Newsdesk, Breitbart, Truth Broadcast Network)
    * **Public dataset** from:

      * Ahmed, H., Traore, I., & Saad, S. (2017): "Detection of Online Fake News Using N-Gram Analysis and Machine Learning Techniques" *(Springer LNCS 10618)*



## **Requirement**

A team consisting of three members must complete a project that involves applying the methods learned from the beginning of the course up to the present. The team is expected to follow and document the entire machine learning workflow, which includes the following steps:

1. **Data Preprocessing**: Clean and prepare the dataset,etc.

2. **Exploratory Data Analysis (EDA)**: Explore and visualize the data.

3. **Model Building**: Select and build one or more machine learning models suitable for the problem at hand.

4. **Hyperparameter set up**: Set and adjust the model's hyperparameters using appropriate methods to improve performance.

5. **Model Training**: Train the model(s) on the training dataset.

6. **Performance Evaluation**: Evaluate the trained model(s) using appropriate metrics (e.g., accuracy, precision, recall, F1-score, confusion matrix, etc.) and validate their performance on unseen data.

7. **Conclusion**: Summarize the results, discuss the model's strengths and weaknesses, and suggest possible improvements or future work.





In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"C:\Users\Admin\OneDrive - VNU-HCMUS\cleaned")
print(df.head())
print(df.shape)

                                                text  label  text_length
0  link imo happening guys jumping conclusions co...      1         2717
1  house passed budget today slim margin democrat...      1         3203
2  youtube video posted january cloudgate studio ...      0         2549
3  marseille france sang danced chanted even reas...      0         8149
4  sunday nbc meet press discussing president don...      0         1235
(68604, 3)


In [3]:
df = df.dropna(subset=['text'])

In [4]:
train_df = df[:int(0.7 * len(df))]
valid_df = df[int(0.7 * len(df)):int(0.8 * len(df))]
test_df = df[int(0.8 * len(df)):]

# Tokenize data

In [5]:
# Tokenize dữ liệu
from transformers import BertTokenizerFast
from transformers import XLNetTokenizerFast

class Tokenizer:
    def __init__(self, model_name, max_length=256):
        if 'bert' in model_name:
            self.tokenizer = BertTokenizerFast.from_pretrained(model_name)
        elif 'xlnet' in model_name:
            self.tokenizer = XLNetTokenizerFast.from_pretrained(model_name)
        self.max_length = max_length

    def tokenize_function(self, examples):
        return self.tokenizer(
            examples["text"],
            truncation=True,
            padding="max_length",
            max_length=self.max_length
        )

from datasets import Dataset

tokenizer = Tokenizer('bert-base-uncased')

# Convert DataFrames to HuggingFace Datasets
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize datasets
train_dataset = train_dataset.map(tokenizer.tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenizer.tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenizer.tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
valid_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/48017 [00:00<?, ? examples/s]

Map:   0%|          | 0/6860 [00:00<?, ? examples/s]

Map:   0%|          | 0/13720 [00:00<?, ? examples/s]

# Build and Load model

In [ ]:
from transformers import BertForSequenceClassification, XLNetForSequenceClassification

class ModelBuilder:
    def __init__(self, num_labels=2):
        self.num_labels = num_labels
        self.model = None

    def build_model(self, model_name):
        if 'bert' in model_name:
            self.model = BertForSequenceClassification.from_pretrained(model_name, num_labels=self.num_labels)
        elif 'xlnet' in model_name:
            self.model = XLNetForSequenceClassification.from_pretrained(model_name, num_labels=self.num_labels)
        return self.model

In [ ]:
model_builder = ModelBuilder()
bert_model = model_builder.build_model('bert-base-uncased')
xlnet_model = model_builder.build_model('xlnet-base-cased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Train model

In [9]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
xlnet_model.to(device)

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0-11): 12 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (sequence_summary): SequenceSummary(
    (summary): Linear(in_features=768, out_features=768, bias=True)
    (activation): Tanh()
    (first_dropout): Identity()
    (last

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

actual_tokenizer = tokenizer.tokenizer

data_collator = DataCollatorWithPadding(tokenizer=actual_tokenizer)

training_args = TrainingArguments(
    output_dir='./bert_results',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    dataloader_num_workers=8,
    fp16=True
)

trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=actual_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/6003 [00:00<?, ?it/s]

{'loss': 0.4713, 'grad_norm': 14.095157623291016, 'learning_rate': 4.587706146926537e-05, 'epoch': 0.08}
{'loss': 0.4065, 'grad_norm': 1.1015597581863403, 'learning_rate': 4.171247709478594e-05, 'epoch': 0.17}
{'loss': 0.3678, 'grad_norm': 166.54788208007812, 'learning_rate': 3.7556221889055474e-05, 'epoch': 0.25}
{'loss': 0.3093, 'grad_norm': 44.07796096801758, 'learning_rate': 3.339163751457605e-05, 'epoch': 0.33}
{'loss': 0.3086, 'grad_norm': 0.5351744890213013, 'learning_rate': 2.9227053140096622e-05, 'epoch': 0.42}
{'loss': 0.2697, 'grad_norm': 7.421328544616699, 'learning_rate': 2.5062468765617193e-05, 'epoch': 0.5}
{'loss': 0.295, 'grad_norm': 38.94197463989258, 'learning_rate': 2.0897884391137763e-05, 'epoch': 0.58}
{'loss': 0.2467, 'grad_norm': 0.4090960919857025, 'learning_rate': 1.6741629185407298e-05, 'epoch': 0.67}
{'loss': 0.2655, 'grad_norm': 18.719730377197266, 'learning_rate': 1.2577044810927872e-05, 'epoch': 0.75}
{'loss': 0.2557, 'grad_norm': 56.69921112060547, 'lear

  0%|          | 0/858 [00:00<?, ?it/s]

{'eval_loss': 0.1998661905527115, 'eval_runtime': 84.2181, 'eval_samples_per_second': 81.455, 'eval_steps_per_second': 10.188, 'epoch': 1.0}
{'train_runtime': 1741.9, 'train_samples_per_second': 27.566, 'train_steps_per_second': 3.446, 'train_loss': 0.3019297282417357, 'epoch': 1.0}


TrainOutput(global_step=6003, training_loss=0.3019297282417357, metrics={'train_runtime': 1741.9, 'train_samples_per_second': 27.566, 'train_steps_per_second': 3.446, 'total_flos': 6316901772610560.0, 'train_loss': 0.3019297282417357, 'epoch': 1.0})

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate(eval_dataset=test_dataset)
print(f"Evaluation results: {eval_results}")

  0%|          | 0/1715 [00:00<?, ?it/s]

Evaluation results: {'eval_loss': 0.2187958061695099, 'eval_runtime': 139.296, 'eval_samples_per_second': 98.495, 'eval_steps_per_second': 12.312, 'epoch': 1.0}
